# Launch Sites Locations Analysis with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

Let's first import required Python packages for this lab:



In [41]:
import folium
import pandas as pd

# Import Folium MarkerCluster Plugin
from folium.plugins import MarkerCluster

# Import Folium MousePosition Plugin
from folium.plugins import MousePosition

# Import Folium DivIcon plugin
from folium.features import DivIcon


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the CSV file using requests
response = requests.get(URL)
response.raise_for_status()  # Check for errors

# Read CSV content into DataFrame
spacex_df = pd.read_csv(io.StringIO(response.text))
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site.


In [43]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_sites_df = launch_sites_df[["Launch Site", "Lat", "Long"]]
launch_sites_df


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [44]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [45]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000,
                       color="#d35400", fill=True).add_child(folium.Popup("NASA Johnson Space Center"))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create and icon as a Text Label
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
)
site_map.add_child(circle)
site_map.add_child(marker)
site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [46]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add Circle + Text label marker
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    coordinate = [lat, lng]
    
    # Circle marker
    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(label))
    
    # Text label (DivIcon)
    marker = folium.Marker(
        location=coordinate,
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{label}</b></div>'
        )
    )
    
    # Add to map
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


#### Geographic Insights: U.S. Launch Site Locations

- **Proximity to the Equator**:  
  None of the launch sites are located near the Equator. They are all situated in the continental United States, which lies significantly north of the Equator. However, the Florida-based sites (CCAFS SLC-40 and CCSFS SLC-41) are relatively closer to the Equator compared to VAFB SLC-4E in California, offering slight advantages in launch efficiency for equatorial orbits.

- **Proximity to the Coast**:  
  All three launch sites are located **very close to coastlines**. This coastal positioning is strategic:
  - It allows rockets to launch over open water, minimizing risk to populated areas.
  - It provides logistical advantages for transporting large payloads and equipment.
  - Coastal sites also support a wider range of orbital inclinations due to unobstructed launch paths.

#### Summary
The launch sites are **not equatorial**, but their **coastal proximity** is a deliberate choice to enhance safety, flexibility, and mission efficiency.

## Task 2: Mark the success/failed launches for each site on the map


In [47]:
# Create a base map centered roughly around all sites
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()],
                      zoom_start=4)

# Add a Marker Cluster so overlapping markers are grouped nicely
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each launch record in spacex_df
for index, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site = row['Launch Site']
    outcome = row['class']   # 1 = success, 0 = failure
    
    # Set color based on success/failure
    color = 'green' if outcome == 1 else 'red'
    
    # Add a small circle marker for each launch
    folium.Circle(
        location=[lat, lon],
        radius=60,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Site: {site}<br>Outcome: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [48]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Let's first create a `MarkerCluster` object


In [49]:
marker_cluster = MarkerCluster()

In [50]:
# Create a new column 'marker_color' based on 'class' column
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x==1 else "red")

# Check the result
spacex_df[["class", "marker_color"]].head()

,class,marker_color
0,0,red
1,0,red
2,0,red
3,0,red
4,0,red


In [51]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Create and add a Marker with customized icon color
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # coordinates
        popup=record['Launch Site'],                 # optional popup info
        icon=folium.Icon(color='white',
                        icon_color=record['marker_color'],
                        icon='rocket',
                        prefix='fa')
    )
    marker_cluster.add_child(marker)

# Display map
site_map



- KSC LC-39A and CCAFS SLC-40 →  green markers, indicating high landing success.

- VAFB SLC-4E → often a mix or fewer total launches.

- Early sites or test sites → more red markers, showing initial failed attempts.

## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [52]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = "topright",
    seperator = "Long:",
    emptry_string = "NaN",
    lng_first = False,
    num_digits = 20,
    prefix = "Lat:",
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [53]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate Radius for each in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [54]:
# Initialize the map centered on one launch site (e.g., CCAFS)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

site_map = folium.Map(location = 
                    [launch_site_lat, launch_site_lon],
                    zoom_start = 10)

# Add Mouse Position
MousePosition(
    position = "topright",
    seperator = " | ",
    prefix = "Coordinates:",
    num_digits = 6
).add_to(site_map)

site_map

In [58]:
# Calculate the actual numeric distance by CALLING the function
distance = calculate_distance(launch_site_lat, launch_site_lon, 28.5630, -80.5745)

coordinate = [28.5630, -80.5745]

distance_marker = folium.Marker(
    coordinate,  # Replace with [latitude, longitude] of the midpoint or your chosen label point
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    )
)
site_map.add_child(distance_marker)

 Draw a `PolyLine` between a launch site to the selected coastline point


In [61]:
# Define the Co-ordinates for the line
coordinates = [
    [launch_site_lat, launch_site_lon],
    [28.5630, -80.5745]
]

lines = folium.PolyLine(locations=coordinates,
                        weight=1, color="blue")

site_map.add_child(lines)

In [67]:
# Launch Site Co-ordinates (CCAFS)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates for nearby features
city_lat, city_lon = 28.4101, -80.6042
railway_lat, railway_lon = 28.5637, -80.5700
highway_lat, highway_lon = 28.5630, -80.5850

# Calculate distances using function
distance_city = calculate_distance(launch_site_lat, launch_site_lon,
                                city_lat, city_lon)

distance_railway = calculate_distance(launch_site_lat, launch_site_lon,
                                    railway_lat, railway_lon)

distance_highway = calculate_distance(launch_site_lat, launch_site_lon,
                                    highway_lat, highway_lon)

# Initialize the map
site_map = folium.Map(location = 
                    [launch_site_lat, launch_site_lon],
                    zoom_start = 11)

# Add the Launch Site Marker

# City marker + line + label
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup = "Launch Site",
    icon = folium.Icon(color="red", icon="rocket")
).add_to(site_map)

# Create Marker for city, railway, highway
folium.Marker([city_lat, city_lon],
            popup="Closest City",
            icon = folium.Icon(color="Blue", icon="building")).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon],
                [city_lat, city_lon]], color = "blue",
                weights=2).add_to(site_map)
folium.Marker(
    [(launch_site_lat + city_lat)/2, (launch_site_lon + city_lon)/2],
    icon=DivIcon(html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_city:.2f} KM</b></div>')
).add_to(site_map)

# Railway marker + line + label
folium.Marker([railway_lat, railway_lon],
            popup="Closest Railway",
            icon = folium.Icon(color="green",icon="road")).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon],
                [railway_lat, railway_lon]],
                color='green',
                weight=2).add_to(site_map)

folium.Marker(
    [(launch_site_lat + railway_lat)/2, (launch_site_lon + railway_lon)/2],
    icon=DivIcon(html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_railway:.2f} KM</b></div>')
).add_to(site_map)

# Highway marker + line + label
folium.Marker([highway_lat, highway_lon], popup='Closest Highway', icon=folium.Icon(color='purple', icon='road')).add_to(site_map)
folium.PolyLine([[launch_site_lat, launch_site_lon],
                [highway_lat, highway_lon]],
                color='purple',
                weight=2).add_to(site_map)
folium.Marker(
    [(launch_site_lat + highway_lat)/2, (launch_site_lon + highway_lon)/2],
    icon=DivIcon(html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_highway:.2f} KM</b></div>')
).add_to(site_map)

# Show the map
site_map

C:\Users\Prashanth\AppData\Local\Temp\ipykernel_19268\660093400.py:37: UserWarning: color argument of Icon should be one of: {'green', 'darkgreen', 'darkblue', 'black', 'blue', 'white', 'lightgreen', 'lightblue', 'orange', 'darkred', 'lightgray', 'lightred', 'purple', 'darkpurple', 'red', 'pink', 'gray', 'cadetblue', 'beige'}.
  icon = folium.Icon(color="Blue", icon="building")).add_to(site_map)


### Proximity Analysis of SpaceX Launch Sites

After plotting distance lines from each launch site to nearby features, we can answer the following spatial questions:

#### 1. Are Launch Sites in Close Proximity to Railways?
-  **Yes**, most launch sites are within a few kilometers of railway lines.
-  *Explanation*: Railways provide efficient transport for heavy equipment and rocket components, making them a strategic logistical asset.

#### 2. Are Launch Sites in Close Proximity to Highways?
-  **Yes**, highways are typically located nearby.
-  *Explanation*: Highways ensure rapid ground access for personnel, materials, and emergency services, supporting operational efficiency.

#### 3. Are Launch Sites in Close Proximity to Coastline?
-  **Yes**, launch sites are often situated near coastlines.
-  *Explanation*: Coastal locations allow rockets to launch over open water, minimizing risk to populated areas and optimizing flight trajectories.

#### 4. Do Launch Sites Keep a Safe Distance from Cities?
-  **Yes**, cities are generally located more than 10 km away.
-  *Explanation*: This buffer zone reduces risks from accidents, noise, and pollution, and complies with safety regulations.

---

#### Summary of Findings
SpaceX launch sites are strategically positioned to balance **logistical access** (railways, highways) with **safety and trajectory needs** (coastlines, urban buffers). This spatial configuration reflects thoughtful planning aligned with both engineering and regulatory priorities.

## Next Steps:

Now we have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, we will need to build a dashboard using Ploty Dash on detailed launch records.

----